# 01 – Visión general del proyecto y lectura del MANIFEST

Notebook de entrada del TFM de scRNA-seq en cirrosis.  
Sirve como ficha técnica del proyecto y del objeto `.h5ad` de entrada.  
No modifica datos biológicos; solo configura rutas, lee el MANIFEST y describe el dataset.

## 1. Tipo de datos con los que trabajaremos

- Objeto principal: `TFM_CIRRHOSIS_merged.h5ad` (AnnData, scRNA-seq: células × genes).
- Archivos auxiliares en `AI_Package/`:
  - Scripts del pipeline (`scripts/*_min.py`).
  - Contexto y documentación (markers, atlas, equivalencias, etc.).
- En los siguientes notebooks se hará QC, integración y anotación; aquí solo se documenta el punto de partida.

In [1]:
# Importamos las bibliotecas básicas que usaremos en este notebook.
from pathlib import Path
import json

import scanpy as sc
import anndata as ad

print("Scanpy:", sc.__name__)
print("AnnData:", ad.__name__)

Scanpy: scanpy
AnnData: anndata


## 2. Organización provisional de carpetas del proyecto

- Se asume que este notebook está en `notebooks/`.
- A partir de ahí se definen:
  - `PROJECT_ROOT` = carpeta raíz del proyecto.
  - `AI_PACKAGE_DIR` = `PROJECT_ROOT / "AI_Package"`.
  - `DATA_RAW_DIR`   = `PROJECT_ROOT / "data_raw"`.
- Se comprueba en pantalla si existen `AI_Package/` y `data_raw/`.

In [2]:
# Detectamos la carpeta raíz del proyecto asumiendo que este notebook está en "notebooks/"
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

AI_PACKAGE_DIR = PROJECT_ROOT / "AI_Package"
DATA_RAW_DIR   = PROJECT_ROOT / "data_raw"  # carpeta recomendada para datos pesados

print("Directorio del notebook:", NOTEBOOK_DIR)
print("Raíz del proyecto:", PROJECT_ROOT)
print("Carpeta AI_Package:", AI_PACKAGE_DIR)
print("Carpeta data_raw:", DATA_RAW_DIR)

print("\n¿Existe AI_Package?:", AI_PACKAGE_DIR.exists())
print("¿Existe data_raw?:   ", DATA_RAW_DIR.exists())

Directorio del notebook: D:\Users\Coni\Documents\TFM_CirrhosIS\notebooks
Raíz del proyecto: D:\Users\Coni\Documents\TFM_CirrhosIS
Carpeta AI_Package: D:\Users\Coni\Documents\TFM_CirrhosIS\AI_Package
Carpeta data_raw: D:\Users\Coni\Documents\TFM_CirrhosIS\data_raw

¿Existe AI_Package?: True
¿Existe data_raw?:    True


## 3. Lectura del MANIFEST del proyecto

- Se carga `AI_Package/MANIFEST.json` y se inspeccionan las claves principales:
  - `scripts`, `data_files`, `context_files`, `keys`.
- Se listan los scripts declarados con su ruta relativa dentro de `AI_Package/`.
- Se comprueba que todas las rutas de `scripts`, `data_files` y `context_files`
  existen realmente bajo `AI_Package/`; si faltara algo, se reportaría en pantalla.
- De la sección `keys` se obtienen los nombres estándar de embeddings y vecinos.

In [3]:
manifest_path = AI_PACKAGE_DIR / "MANIFEST.json"

if not manifest_path.exists():
    raise FileNotFoundError(f"No se encuentra MANIFEST.json en {manifest_path}")

with open(manifest_path, "r", encoding="utf-8") as f:
    manifest = json.load(f)

# Mostramos las claves de primer nivel para hacernos una idea de la estructura
manifest.keys()

dict_keys(['project_name', 'version', 'keys', 'entrypoints', 'package_role', 'external_dependencies', 'dependencies_min', 'scripts', 'pipelines', 'exclusions', 'context_files', 'data_files', 'expects', 'notes'])

In [4]:
# Mostramos secciones importantes del MANIFEST de forma resumida

print("=== Secciones principales del MANIFEST ===")
for key in ["scripts", "data_files", "context_files", "keys"]:
    if key in manifest:
        value = manifest[key]
        if isinstance(value, dict):
            type_str = f"dict ({len(value)} entradas)"
        elif isinstance(value, list):
            type_str = f"list ({len(value)} elementos)"
        else:
            type_str = type(value).__name__
        print(f"- {key}: {type_str}")

print("\n=== Claves estándar (keys) ===")
for k, v in manifest.get("keys", {}).items():
    print(f"{k}: {v}")

print("\n=== Scripts declarados ===")
scripts_section = manifest.get("scripts", {})

if isinstance(scripts_section, dict):
    # Caso: scripts = { "nombre_script": { "path": "...", ... }, ... }
    for name, entry in scripts_section.items():
        if isinstance(entry, dict):
            path = entry.get("path", "<sin path>")
            desc = entry.get("description", "")
        else:
            path = str(entry)
            desc = ""
        if desc:
            print(f"- {name}: {path}  ({desc})")
        else:
            print(f"- {name}: {path}")
elif isinstance(scripts_section, list):
    # Caso alternativo: scripts = [ { "name": "...", "path": "..." }, ... ]
    for entry in scripts_section:
        if isinstance(entry, dict):
            name = entry.get("name", "<sin nombre>")
            path = entry.get("path", "<sin path>")
            desc = entry.get("description", "")
            if desc:
                print(f"- {name}: {path}  ({desc})")
            else:
                print(f"- {name}: {path}")
        else:
            print(f"- <entrada no dict>: {entry}")
else:
    print(f"[AVISO] Formato inesperado para 'scripts': {type(scripts_section).__name__}")

=== Secciones principales del MANIFEST ===
- scripts: dict (12 entradas)
- data_files: dict (4 entradas)
- context_files: list (6 elementos)
- keys: dict (3 entradas)

=== Claves estándar (keys) ===
EMB_KEY: X_pca_harmony
NBR_KEY: harmony
UMAP_KEY: X_umap_harmony

=== Scripts declarados ===
- build_harmony: scripts/00_harmony_embedding/00_build_harmony_embedding.py
- global_clustering: scripts/02_lineages_l1/11_lineages_clustering_min.py
- annotate_lineages_l1: scripts/02_lineages_l1/12_lineages_annotation_min.py
- split_by_lineage: scripts/02_lineages_l1/13_lineages_split_min.py
- per_lineage_qc_umap_leiden: scripts/03_celltypes_l2/21_per_lineage_qc_hvg_umap_leiden_min.py
- per_lineage_markers: scripts/03_celltypes_l2/22_per_lineage_markers_min.py
- build_main_annotated: scripts/04_main_assembly_plots/31_build_main_annotated_min.py
- plots_main_annotation: scripts/04_main_assembly_plots/32_plots_main_annotation_min.py
- dotplots: scripts/05_characterization_markers_validation/41_dotpl

In [5]:
# Comprobamos qué archivos declarados en el MANIFEST existen realmente en AI_Package/
missing_paths = []

def _check_relative_path(rel_path: str):
    path = AI_PACKAGE_DIR / rel_path
    if not path.exists():
        missing_paths.append(rel_path)

def _scan_section_entries(section_name: str):
    """
    Dado un nombre de sección del MANIFEST ('scripts', 'data_files', 'context_files'),
    extrae todos los paths relativos que aparezcan ahí y los pasa a _check_relative_path.
    Soporta formatos:
      - dict: {name: { "path": "..." }} o {name: "ruta"}
      - list: ["ruta1", "ruta2"] o [ { "path": "..." }, ... ]
      - str: camino directo
    """
    entries = manifest.get(section_name, {})

    # Caso dict
    if isinstance(entries, dict):
        for name, entry in entries.items():
            if isinstance(entry, dict):
                rel = entry.get("path")
                if rel:
                    _check_relative_path(rel)
            elif isinstance(entry, str):
                _check_relative_path(entry)

    # Caso lista
    elif isinstance(entries, list):
        for entry in entries:
            if isinstance(entry, dict):
                rel = entry.get("path")
                if rel:
                    _check_relative_path(rel)
            elif isinstance(entry, str):
                _check_relative_path(entry)

    # Caso string directo
    elif isinstance(entries, str):
        _check_relative_path(entries)

    else:
        print(f"[AVISO] Formato inesperado para sección '{section_name}': {type(entries).__name__}")

# Revisamos scripts, data_files y context_files
for section in ["scripts", "data_files", "context_files"]:
    _scan_section_entries(section)

print("Rutas declaradas en MANIFEST y no encontradas en AI_Package/:")
if not missing_paths:
    print("- Ninguna: todo lo declarado está presente.")
else:
    for rel in missing_paths:
        print(f"- {rel}")

Rutas declaradas en MANIFEST y no encontradas en AI_Package/:
- Ninguna: todo lo declarado está presente.


## 4. Configuración del archivo de datos principal (`.h5ad`)

- Se define la ruta esperada del archivo principal:
  - `RAW_H5AD_PATH = DATA_RAW_DIR / "TFM_CIRRHOSIS_merged.h5ad"`.
- Se imprime la ruta y se comprueba si el archivo existe.
- Aquí solo se fija la ruta; la lectura del objeto se hace más adelante en el notebook.
- No se aplica ningún preprocesamiento en este punto.

In [6]:
# Ruta esperada del archivo principal .h5ad
# Modifica esto si tu archivo está en otra localización o tiene otro nombre.
RAW_H5AD_PATH = DATA_RAW_DIR / "TFM_CIRRHOSIS_merged.h5ad"

print("Archivo esperado .h5ad principal:")
print(RAW_H5AD_PATH)
print("¿Existe?:", RAW_H5AD_PATH.exists())

Archivo esperado .h5ad principal:
D:\Users\Coni\Documents\TFM_CirrhosIS\data_raw\TFM_CIRRHOSIS_merged.h5ad
¿Existe?: True


(En este notebook no cargamos aún el .h5ad para no consumir memoria de forma innecesaria.)

## 5. Constantes de configuración para el pipeline

- Se crea un diccionario `CONFIG` con:
  - Rutas básicas: `PROJECT_ROOT`, `AI_PACKAGE_DIR`, `DATA_RAW_DIR`, `RAW_H5AD_PATH`.
  - Claves estándar tomadas de `MANIFEST.json` (con valores por defecto si faltan):
    - `EMB_KEY` (embedding Harmony, p.ej. `"X_pca_harmony"`).
    - `NBR_KEY` (grafo de vecinos, p.ej. `"harmony"`).
    - `UMAP_KEY` (UMAP armonizado, p.ej. `"X_umap_harmony"`).
- Esta celda se puede reutilizar en el resto de notebooks para mantener nombres coherentes.

In [7]:
CONFIG = {
    "PROJECT_ROOT": PROJECT_ROOT,
    "AI_PACKAGE_DIR": AI_PACKAGE_DIR,
    "DATA_RAW_DIR": DATA_RAW_DIR,
    "RAW_H5AD_PATH": RAW_H5AD_PATH,
    # Claves estándar tal y como figuran en MANIFEST.json
    "EMB_KEY": manifest.get("keys", {}).get("EMB_KEY", "X_pca_harmony"),
    "NBR_KEY": manifest.get("keys", {}).get("NBR_KEY", "harmony"),
    "UMAP_KEY": manifest.get("keys", {}).get("UMAP_KEY", "X_umap_harmony"),
}

CONFIG

{'PROJECT_ROOT': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS'),
 'AI_PACKAGE_DIR': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/AI_Package'),
 'DATA_RAW_DIR': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/data_raw'),
 'RAW_H5AD_PATH': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/data_raw/TFM_CIRRHOSIS_merged.h5ad'),
 'EMB_KEY': 'X_pca_harmony',
 'NBR_KEY': 'harmony',
 'UMAP_KEY': 'X_umap_harmony'}

In [8]:
# Cargamos el objeto bruto en modo "backed" (lectura en disco, sin cargar toda la matriz en RAM)
raw_path = CONFIG["RAW_H5AD_PATH"]
print("Leyendo objeto bruto desde:", raw_path)

adata_raw = sc.read_h5ad(raw_path, backed="r")
print(adata_raw)

# Resumen de columnas en obs y var
print("\nColumnas en adata_raw.obs:")
print(list(adata_raw.obs.columns))

print("\nColumnas en adata_raw.var:")
print(list(adata_raw.var.columns))

# Algunos metadatos básicos si existen
# Incluimos 'gem_id' porque en el pipeline se usa como batch técnico principal
candidate_sample_cols = ["sample", "sample_id", "libraryID", "gem_id", "batch"]
candidate_condition_cols = ["disease", "condition", "group", "status"]

print("\nResumen de posibles columnas de muestra:")
for col in candidate_sample_cols:
    if col in adata_raw.obs.columns:
        print(f"\nDistribución de {col}:")
        print(adata_raw.obs[col].value_counts())

print("\nResumen de posibles columnas de condición:")
for col in candidate_condition_cols:
    if col in adata_raw.obs.columns:
        print(f"\nDistribución de {col}:")
        print(adata_raw.obs[col].value_counts())

Leyendo objeto bruto desde: D:\Users\Coni\Documents\TFM_CirrhosIS\data_raw\TFM_CIRRHOSIS_merged.h5ad
AnnData object with n_obs × n_vars = 231953 × 38606 backed at 'D:\\Users\\Coni\\Documents\\TFM_CirrhosIS\\data_raw\\TFM_CIRRHOSIS_merged.h5ad'
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'gem_id', 'patientID', 'age', 'sex', 'diagnostic', 'disease', 'disease_classification', 'disease_status', 'disease_grade', 'alternative_classification', 'comorbidity', 'sample_collection', 'scrublet_doublet_scores', 'scrublet_predicted_doublet'
    var: 'features'

Columnas en adata_raw.obs:
['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'gem_id', 'patientID', 'age', 'sex', 'diagnostic', 'disease', 'disease_classification', 'disease_status', 'disease_grade', 'alternative_classification', 'comorbidity', 'sample_collection', 'scrublet_doublet_scores', 'scrublet_predicted_doublet']

Columnas en adata_raw.var:
['features']

Resumen de posibles columnas de muestra:

Distribución de gem_id:
gem_id
ee31

In [9]:
# Inspeccionamos cómo están guardados los genes
print("Primeros 10 var_names (índice de adata_raw.var):")
print(list(adata_raw.var_names[:10]))

if "features" in adata_raw.var.columns:
    print("\nPrimeros 10 valores de var['features']:")
    print(list(adata_raw.var["features"][:10]))
else:
    print("\nNo existe la columna 'features' en adata_raw.var.")

Primeros 10 var_names (índice de adata_raw.var):
['DDX11L2', 'MIR1302-2HG', 'FAM138A', 'ENSG00000290826', 'OR4F5', 'ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599']

Primeros 10 valores de var['features']:
['DDX11L2', 'MIR1302-2HG', 'FAM138A', 'ENSG00000290826', 'OR4F5', 'ENSG00000238009', 'ENSG00000239945', 'ENSG00000239906', 'ENSG00000241860', 'ENSG00000241599']


Dejamos el notebook 01 ya con una mini descripción del dataset para la memoria.

Esto no modifica datos, solo describe; deja el notebook 01 como una ficha técnica del .h5ad de entrada.

In [10]:
# Nota: aquí SOLO resumimos scrublet_predicted_doublet, no filtramos por doublets.
# Resumen de pacientes
if "patientID" in adata_raw.obs.columns:
    print("Número de pacientes únicos:", adata_raw.obs["patientID"].nunique())
    print("\nDistribución de patientID (top 10):")
    print(adata_raw.obs["patientID"].value_counts().head(10))

# Resumen de scrublet
if "scrublet_predicted_doublet" in adata_raw.obs.columns:
    print("\nDistribución de scrublet_predicted_doublet:")
    print(adata_raw.obs["scrublet_predicted_doublet"].value_counts())

Número de pacientes únicos: 16

Distribución de patientID (top 10):
patientID
CNAG_118    38172
CNAG_117    36220
CNAG_121    19747
CNAG_123    12832
IJC_01      12803
CNAG_143    12565
CNAG_126    12202
IJC_03      12115
CNAG_124    11398
CNAG_125    10838
Name: count, dtype: int64

Distribución de scrublet_predicted_doublet:
scrublet_predicted_doublet
False    149621
NA        74392
True       7940
Name: count, dtype: int64


## 5. Resumen del objeto de entrada (`TFM_CIRRHOSIS_merged.h5ad`)

- Se carga el archivo con `sc.read_h5ad(..., backed="r")`:
  - ~231 953 células × 38 606 genes (lectura en disco, sin cargar toda la matriz en RAM).
- Metadatos celulares (`adata_raw.obs`, ejemplos):
  - Calidad / técnica: `orig.ident`, `nCount_RNA`, `nFeature_RNA`, `gem_id`.
  - Clínicos: `patientID`, `age`, `sex`, `diagnostic`, `disease`,
    `disease_classification`, `disease_status`, `disease_grade`,
    `alternative_classification`, `comorbidity`, `sample_collection`.
  - Scrublet: `scrublet_doublet_scores`, `scrublet_predicted_doublet`.
- Metadatos de genes (`adata_raw.var`):
  - Columna `features`; los primeros valores coinciden con `var_names`
    y se usan como nombres de genes.
- Resúmenes básicos:
  - Pacientes: 16 `patientID` con números de células desiguales.
  - `disease`:
    - Cirrhosis: 153 974 células.
    - Healthy: 77 979 células.
  - `scrublet_predicted_doublet`:
    - True: 7 940 células.
    - False: 149 621 células.
    - NA: 74 392 células.
- En este notebook solo se describen estos campos; **no se filtran doublets ni se modifican los datos**.

## 6. Resumen de este notebook

- Define la estructura de carpetas del proyecto y comprueba la existencia de `AI_Package/` y `data_raw/`.
- Lee `MANIFEST.json`, lista sus secciones principales y valida las rutas de scripts y datos.
- Construye `CONFIG` con rutas y claves estándar (`EMB_KEY`, `NBR_KEY`, `UMAP_KEY`).
- Localiza y carga `TFM_CIRRHOSIS_merged.h5ad` en modo `backed="r"` y resume:
  dimensiones, metadatos clave, pacientes, condición clínica y Scrublet.
- Deja documentado el objeto de entrada y el “contrato” técnico del pipeline,
  sin aplicar todavía QC, normalización ni anotaciones biológicas.